In [2]:
!pip install -q torch==2.9.0 cuda-bindings==12.9.5 unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 122.8 MB/s eta 0:00:00


In [4]:
!pip install -q trl peft accelerate bitsandbytes

In [8]:
!pip install -q "opentelemetry-api<=1.37.0" "opentelemetry-sdk<=1.37.0" deepeval datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.6 MB/s eta 0:00:00


In [9]:
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

GPU: NVIDIA A100-SXM4-80GB
Memory: 85.2 GB


# 1. Load Model with UnSloth

In [11]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-0.5B-Instruct",
    max_seq_length=512,
    load_in_4bit=False,
    load_in_8bit=True,
)

==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

# 2. Applying LoRA

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing="unsloth",
)

# Trainable Params Count
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Total Model Params: {total:,}\nTrainable params: {100*trainable/total:.2f}%")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients
Total Trainable params:
	 Total Model Params: 502,830,976
	 Trainable params: 1.75%


# 3. Preparing Data for Fine-Tuning

In [13]:
from datasets import Dataset

# Dummy data
data = [{"text": tokenizer.apply_chat_template([
    {"role": "user", "content": f"What is {i}+{i}?"},
    {"role": "assistant", "content": str(i*2)}
], tokenize=False)} for i in range(1, 21)]

dataset = Dataset.from_list(data)
print(f"Samples: {len(dataset)}")
print(f"Example:\n{dataset[0]['text'][:200]}...")

Samples: 20
Example:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is 1+1?<|im_end|>
<|im_start|>assistant
2<|im_end|>
...


# 4. Dummy Training

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=TrainingArguments(
        output_dir="./test_output",
        max_steps=10,
        per_device_train_batch_size=2,
        logging_steps=5,
        optim="adamw_8bit",
        bf16=True,
        report_to="none",
    ),
    max_seq_length=512,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/20 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 8,798,208 of 502,830,976 (1.75% trained)


Step,Training Loss
5,3.339400
10,1.901600


TrainOutput(global_step=10, training_loss=2.6205183982849123, metrics={'train_runtime': 46.9449, 'train_samples_per_second': 0.426, 'train_steps_per_second': 0.213, 'total_flos': 1830548029440.0, 'train_loss': 2.6205183982849123, 'epoch': 1.0})

# 5. Test Inference

In [16]:
FastLanguageModel.for_inference(model)

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": "What is 5+5?"}],
    tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens=32, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
What is 5+5?
assistant
10


# 6. Model test using DeepEval

In [17]:
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

print("DeepEval imported successfully!")

DeepEval imported successfully!


# 7. Deleting test file

In [18]:
!rm -rf ./test_output/